### Implementing FunkSVD

In this notebook we will take a look at writing our own function that performs FunkSVD, which will follow the steps you saw in the previous video.  If you find that you aren't ready to tackle this task on your own, feel free to skip to the following video where you can watch as I walk through the steps.

To test our algorithm, we will run it on the subset of the data you worked with earlier.  Run the cell below to get started.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import sparse
import svd_tests as t
%matplotlib inline

# Read in the datasets
movies = pd.read_csv('data/movies_clean.csv')
reviews = pd.read_csv('data/reviews_clean.csv')

del movies['Unnamed: 0']
del reviews['Unnamed: 0']

# Create user-by-item matrix
user_items = reviews[['user_id', 'movie_id', 'rating', 'timestamp']]
user_by_movie = user_items.groupby(['user_id', 'movie_id'])['rating'].max().unstack()

# Create data subset
user_movie_subset = user_by_movie[[73486, 75314,  68646, 99685]].dropna(axis=0)
ratings_mat = np.matrix(user_movie_subset)
print(ratings_mat)

[[10. 10. 10. 10.]
 [10.  4.  9. 10.]
 [ 8.  9. 10.  5.]
 [ 9.  8. 10. 10.]
 [10.  5.  9.  9.]
 [ 6.  4. 10.  6.]
 [ 9.  8. 10.  9.]
 [10.  5.  9.  8.]
 [ 7.  8. 10.  8.]
 [ 9.  5.  9.  7.]
 [ 9.  8. 10.  8.]
 [ 9. 10. 10.  9.]
 [10.  9. 10.  8.]
 [ 5.  8.  5.  8.]
 [10.  8. 10. 10.]
 [ 9.  9. 10. 10.]
 [ 9.  8.  8.  8.]
 [10.  8.  1. 10.]
 [ 5.  6. 10. 10.]
 [ 8.  7. 10.  7.]]


`1.` You will use the **user_movie_subset** matrix to show that your FunkSVD algorithm will converge.  In the below cell, use the comments and document string to assist you as you complete writing your own function to complete FunkSVD.  You may also want to try to complete the function on your own without the assistance of comments.  You may feel free to remove and add to the function in any way that gets you a working solution! 

**Notice:** There isn't a sigma matrix in this version of the matrix factorization.

In [37]:
n_users = ratings_mat.shape[0] # number of rows in the matrix
n_movies = ratings_mat.shape[1] # number of movies in the matrix
num_ratings = np.sum(~np.isnan(ratings_mat))
user_mat = np.random.rand(n_users, 4) # user matrix filled with random values of shape user x latent 
movie_mat = np.random.rand(4, n_movies)
dotporduct=np.dot(user_mat,movie_mat)
np.sum(~np.isnan(dotporduct))

80

In [48]:
ratings_mat[1,2]

9.0

In [81]:
[i for i in user_mat[1,:]]

[0.38680822688825217,
 0.8541261994080624,
 1.5188245335167896,
 1.2000438017292547]

In [82]:
def FunkSVD(ratings_mat, latent_features=4, learning_rate=0.0001, iters=100):
    '''
    This function performs matrix factorization using a basic form of FunkSVD with no regularization
    
    INPUT:
    ratings_mat - (numpy array) a matrix with users as rows, movies as columns, and ratings as values
    latent_features - (int) the number of latent features used
    learning_rate - (float) the learning rate 
    iters - (int) the number of iterations
    
    OUTPUT:
    user_mat - (numpy array) a user by latent feature matrix
    movie_mat - (numpy array) a latent feature by movie matrix
    '''
    
    # Set up useful values to be used through the rest of the function
    n_users = ratings_mat.shape[0] # number of rows in the matrix
    n_movies = ratings_mat.shape[1] # number of movies in the matrix
    num_ratings = np.sum(~np.isnan(ratings_mat)) # total number of ratings in the matrix
    
    # initialize the user and movie matrices with random values
    # helpful link: https://docs.scipy.org/doc/numpy/reference/generated/numpy.random.rand.html
    user_mat = np.random.rand(n_users, latent_features) # user matrix filled with random values of shape user x latent 
    movie_mat = np.random.rand(latent_features, n_movies)# movie matrix filled with random values of shape latent x movies
    
    # initialize sse at 0 for first iteration
    sse_accum = 0
    
    # header for running results
    print("Optimization Statistics")
    print("Iterations | Mean Squared Error ")
    
    # for each iteration
    for i in range(iters):
        # update our sse
        old_sse = sse_accum
        sse_accum = 0
        
        # For each user-movie pair
        for u in range(n_users):
            for m in range(n_movies):
                # if the rating exists
                if ratings_mat[u,m] ==ratings_mat[u,m]:
                    # compute the error as the actual minus the dot product of the user and movie latent features
                    error= ratings_mat[u,m]-np.dot(user_mat[u,:],movie_mat[:,m])
                    # Keep track of the total sum of squared errors for the matrix
                    sse_accum+=error**2
                    # update the values in each matrix in the direction of the gradient
                    for l in range(latent_features):
                        user_mat[u,l]+=learning_rate*2*error*movie_mat[l,m]
                        movie_mat[l,m]+= learning_rate*2*error*user_mat[u,l]
        # print results for iteration
        print(i+1,"\t|",sse_accum/num_ratings)
    return user_mat, movie_mat 

`2.` Try out your function on the **user_movie_subset** dataset.  First try 4 latent features, a learning rate of 0.005, and 10 iterations.  When you take the dot product of the resulting U and V matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?

In [83]:
# use your function with 4 latent features, lr of 0.005 and 10 iterations
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4,learning_rate=0.005, iters=10)

Optimization Statistics
Iterations | Mean Squared Error 
1 	| 42.61048838891536
2 	| 14.747454456595014
3 	| 4.118798712433147
4 	| 2.8402305904139435
5 	| 2.7060564763342914
6 	| 2.655403713474746
7 	| 2.6138488095390744
8 	| 2.570472505759626
9 	| 2.5224628221822143
10 	| 2.468675144634086


In [84]:
#Compare the predicted and actual results
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[ 9.22181889  8.98579927 10.4697708  10.2658572 ]
 [ 8.74703048  7.56979183  9.43621413  8.34826072]
 [ 7.82076477  6.87804006  8.49323128  7.5989622 ]
 [ 9.28840601  8.44459616 10.23653607  9.47024616]
 [ 8.52761475  7.44191372  8.41119704  9.01373906]
 [ 7.05249805  5.93735967  7.12110639  6.85770062]
 [ 9.03758604  8.16967307  9.58986031  9.29558646]
 [ 8.37730706  7.08275727  8.84825624  7.94541504]
 [ 8.5323216   7.38211516  8.61488926  8.73511641]
 [ 8.23144337  6.49663312  7.91834517  7.60805814]
 [ 8.84051053  7.77817616  9.15253909  9.03021351]
 [ 9.54114599  8.40447667 10.16805043  9.49298111]
 [ 9.25988839  8.12125275  9.82715465  9.1513919 ]
 [ 6.26651593  5.92656619  6.72689196  6.94971019]
 [ 9.63313925  8.56908153 10.24234284  9.75448041]
 [ 9.70448171  8.58729669 10.28344506  9.67253244]
 [ 8.70677151  7.09986556  8.61024927  8.20575802]
 [ 7.15319916  6.13929291  6.60809517  7.78537535]
 [ 7.94612595  7.49457593  8.87895335  8.49872245]
 [ 8.43442366  7.06627397  8.97

**Write your summary here.**
SSE after 10 iterates is at 2.5, predict result is getting close to the actual result but still quite far

`3.` Let's try out the function again on the **user_movie_subset** dataset.  This time we will again use 4 latent features and a learning rate of 0.005.  However, let's bump up the number of iterations to 250.  When you take the dot product of the resulting U and V matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?  What do you notice about your error at the end of the 250 iterations?

In [85]:
#use your function with 4 latent features, lr of 0.005 and 250 iterations
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4,learning_rate=0.005, iters=250)

Optimization Statistics
Iterations | Mean Squared Error 
1 	| 42.99141940170008
2 	| 15.012626199445734
3 	| 4.113877927737579
4 	| 2.817128396963048
5 	| 2.718776282769822
6 	| 2.686794480311467
7 	| 2.6578685629635683
8 	| 2.627351407486479
9 	| 2.5941149903647225
10 	| 2.5574882124110645
11 	| 2.5169269090516297
12 	| 2.4719735871093134
13 	| 2.4222677226912377
14 	| 2.3675676301981188
15 	| 2.307775273157888
16 	| 2.2429594575519225
17 	| 2.1733727267582315
18 	| 2.099457129183151
19 	| 2.0218348377639774
20 	| 1.941281649601264
21 	| 1.8586845240859071
22 	| 1.774987872198791
23 	| 1.6911362311260958
24 	| 1.6080221406266602
25 	| 1.52644683055606
26 	| 1.4470979408702147
27 	| 1.3705440228371992
28 	| 1.2972415724379462
29 	| 1.2275481632014666
30 	| 1.161735422702135
31 	| 1.0999977248795787
32 	| 1.0424554712445921
33 	| 0.9891544988351856
34 	| 0.9400646353827563
35 	| 0.8950804825874277
36 	| 0.8540264611429093
37 	| 0.8166666129584222
38 	| 0.7827182463401408
39 	| 0.7518676

In [86]:
#Compare the predicted and actual results
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[ 8.06275566  9.99999635  9.99999832 10.00000353]
 [ 9.99999813  3.99999884  8.99999957 10.00000201]
 [ 7.99999019  8.99999524  9.99999817  5.00000928]
 [ 9.00002429  8.0000125  10.00000517  9.99997672]
 [ 9.99999708  4.99999799  8.99999907  9.00000305]
 [ 6.00003813  4.00001901 10.00000755  5.99996358]
 [ 8.99996963  7.99998423  9.99999339  9.00002885]
 [ 9.99997792  4.99998851  8.99999535  8.00002133]
 [ 7.00003189  8.00001642 10.0000067   7.99996929]
 [ 8.99997899  4.9999891   8.99999559  7.00002025]
 [ 9.00000852  8.00000438 10.00000183  7.99999182]
 [ 9.00001358 10.00000693 10.00000273  8.99998675]
 [ 9.99999928  8.99999948  9.99999967  8.00000057]
 [ 4.99997442  7.99998746  4.99999485  8.00002376]
 [10.00004206  8.00002135 10.00000858  9.99995965]
 [ 8.9999992   8.99999937  9.99999933 10.00000016]
 [ 8.99998404  7.99999187  7.99999648  8.00001483]
 [10.00001289  8.00000689  1.00000293  9.99998763]
 [ 4.99997941  5.99999     9.99999566 10.00001856]
 [ 7.99998266  6.99999178  9.99

**Write your summary here.**

250 iterations, results are almost the same

The last time we placed an **np.nan** value into this matrix the entire svd algorithm in python broke.  Let's see if that is still the case using your FunkSVD function.  In the below cell, I have placed a nan into the first cell of your numpy array.  

`4.` Use 4 latent features, a learning rate of 0.005, and 250 iterations.  Are you able to run your SVD without it breaking (something that was not true about the python built in)?  Do you get a prediction for the nan value?  What is your prediction for the missing value? Use the cells below to answer these questions.

In [87]:
# Here we are placing a nan into our original subset matrix
ratings_mat[0, 0] = np.nan
ratings_mat

matrix([[nan, 10., 10., 10.],
        [10.,  4.,  9., 10.],
        [ 8.,  9., 10.,  5.],
        [ 9.,  8., 10., 10.],
        [10.,  5.,  9.,  9.],
        [ 6.,  4., 10.,  6.],
        [ 9.,  8., 10.,  9.],
        [10.,  5.,  9.,  8.],
        [ 7.,  8., 10.,  8.],
        [ 9.,  5.,  9.,  7.],
        [ 9.,  8., 10.,  8.],
        [ 9., 10., 10.,  9.],
        [10.,  9., 10.,  8.],
        [ 5.,  8.,  5.,  8.],
        [10.,  8., 10., 10.],
        [ 9.,  9., 10., 10.],
        [ 9.,  8.,  8.,  8.],
        [10.,  8.,  1., 10.],
        [ 5.,  6., 10., 10.],
        [ 8.,  7., 10.,  7.]])

In [88]:
# run SVD on the matrix with the missing value
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4,learning_rate=0.005, iters=250)
#use your function with 4 latent features, lr of 0.005 and 250 iterations

Optimization Statistics
Iterations | Mean Squared Error 
1 	| 43.03391196108116
2 	| 14.954045908868173
3 	| 4.186658942354355
4 	| 2.820583179036929
5 	| 2.687967146795337
6 	| 2.647566041636208
7 	| 2.6167629249566287
8 	| 2.5850688664255355
9 	| 2.5498394873731365
10 	| 2.5099708480061924
11 	| 2.4647026875962195
12 	| 2.4133595179652647
13 	| 2.3553106180816172
14 	| 2.289987545337922
15 	| 2.216921998265119
16 	| 2.135795054491361
17 	| 2.0464941769248757
18 	| 1.9491739349840949
19 	| 1.8443141992387544
20 	| 1.732766882556508
21 	| 1.6157799887358366
22 	| 1.4949868806139464
23 	| 1.3723505812857952
24 	| 1.2500586166475174
25 	| 1.1303734261007092
26 	| 1.0154549860205109
27 	| 0.9071824311641786
28 	| 0.8070057978723838
29 	| 0.7158546181013556
30 	| 0.6341174361690715
31 	| 0.5616896123165803
32 	| 0.49807205253967785
33 	| 0.4424955629005631
34 	| 0.3940457723686743
35 	| 0.3517699963042182
36 	| 0.3147563392871245
37 	| 0.2821834306570759
38 	| 0.2533446279264383
39 	| 0.22

In [89]:
# Run this cell to see if you were able to predict for the missing value
preds = np.dot(user_mat, movie_mat)
print("The predicted value for the missing rating is {}:".format(preds[0,0]))
print()
print("The actual value for the missing rating is {}:".format(ratings_mat[0,0]))
print()
assert np.isnan(preds[0,0]) == False
print("That's right! You just predicted a rating for a user-movie pair that was never rated!")
print("But if you look in the original matrix, this was actually a value of 10. Not bad!")

The predicted value for the missing rating is 10.7920211601313:

The actual value for the missing rating is nan:

That's right! You just predicted a rating for a user-movie pair that was never rated!
But if you look in the original matrix, this was actually a value of 10. Not bad!


Now let's extend this to a more realistic example. Unfortunately, running this function on your entire user-movie matrix is still not something you likely want to do on your local machine.  However, we can see how well this example extends to 1000 users.  In the above portion, you were using a very small subset of data with no missing values.

`5.` Given the size of this matrix, this will take quite a bit of time.  Consider the following hyperparameters: 4 latent features, 0.005 learning rate, and 20 iterations.  Grab a snack, take a walk, and this should be done running in a bit.

In [90]:
# Setting up a matrix of the first 1000 users with movie ratings
first_1000_users = np.matrix(user_by_movie.head(1000))

# perform funkSVD on the matrix of the top 1000 users
#fit to 1000 users with 4 latent features, lr of 0.005, and 20 iterations
user_mat, movie_mat = FunkSVD(first_1000_users, latent_features=4,learning_rate=0.005, iters=20)

Optimization Statistics
Iterations | Mean Squared Error 
1 	| 23.34503388536003
2 	| 10.723797866141538
3 	| 7.345322109751491
4 	| 5.673716194415893
5 	| 4.623432493271332
6 	| 3.88748439596206
7 	| 3.3382213271525636
8 	| 2.9114463468383245
9 	| 2.570301145248411
10 	| 2.2917241856211543
11 	| 2.0605285009628442
12 	| 1.866339781130607
13 	| 1.7018142503217089
14 	| 1.5615460282119564
15 	| 1.4413902846026467
16 	| 1.338048550393388
17 	| 1.2488197741833564
18 	| 1.1714533337469901
19 	| 1.1040611132669191
20 	| 1.04506056928122


`6.` Now that you have a set of predictions for each user-movie pair,  let's answer a few questions about your results. Provide the correct values for each of the variables below, and check your solutions using the tests below.

In [92]:
# Replace each of the comments below with the correct values
num_ratings = np.sum(~np.isnan(first_1000_users))
# How many actual ratings exist in first_1000_users
print("The number of actual ratings in the first_1000_users is {}.".format(num_ratings))
print()


ratings_for_missing = np.sum(np.isnan(first_1000_users))
# How many ratings did we make for user-movie pairs that didn't actually have ratings
print("The number of ratings made for user-movie pairs that didn't have ratings is {}".format(ratings_for_missing))

The number of actual ratings in the first_1000_users is 10852.

The number of ratings made for user-movie pairs that didn't have ratings is 31234148


In [93]:
# Test your results against the solution
assert num_ratings == 10852, "Oops!  The number of actual ratings doesn't quite look right."
assert ratings_for_missing == 31234148, "Oops!  The number of movie-user pairs that you made ratings for that didn't actually have ratings doesn't look right."

# Make sure you made predictions on all the missing user-movie pairs
preds = np.dot(user_mat, movie_mat)
assert np.isnan(preds).sum() == 0
print("Nice job!  Looks like you have predictions made for all the missing user-movie pairs! But I still have one question... How good are they?")

Nice job!  Looks like you have predictions made for all the missing user-movie pairs! But I still have one question... How good are they?
